<a href="https://colab.research.google.com/github/tasty-orange/ARN-labs/blob/main/ARN_PW3_PT1_Piemontesi_Trueb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Practical Work 3 ARN - G.Piemontesi & G.Trueb

## Première expérience : Classification « awake » / « asleep »

Dans cette expérience, nous allons classifier les états « awake » (éveillé) et « asleep » (endormi) des souris en regroupant les phases n-rem et rem sous « asleep ». L'objectif est de créer un modèle MLP pour prédire ces deux états à partir des données EEG des souris. Nous évaluerons les performances du modèle à l'aide de la perte d'entraînement, de la validation, de la matrice de confusion et du F1-score.

### Import des packages/dépendances



In [1]:
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Input, Dense
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import confusion_matrix, classification_report, f1_score
from collections import Counter
from sklearn.preprocessing import StandardScaler, LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras import regularizers
from tensorflow import keras

### Import des données des souris

In [2]:
# Chargement des données d'entraînement
mice_eeg_1 = pd.read_csv('EEG_mouse_data_1.csv')
mice_eeg_2 = pd.read_csv('EEG_mouse_data_2.csv')

# Fusion des deux jeux de données d'entraînement
data_training = pd.concat([mice_eeg_1, mice_eeg_2])

FileNotFoundError: [Errno 2] No such file or directory: 'EEG_mouse_data_1.csv'

###Preprocessing et normalisation des données

Comme on peut le voir dans le diagramme en barres ci-dessous, nous avons trois états pour le sommeil des souris, et nous voulons n'en avoir que deux : 'awake' et 'asleep'.

In [ ]:
data_training.state.value_counts().plot(kind='bar', cmap='Dark2')
plt.title('Total Occurences per State')
plt.xticks(rotation=0)
plt.ylabel('Count')
plt.xlabel('State');

Modification du dataset pour fusionner les états 'n' et 'r' en un seul pour 'asleep', afin de n'avoir plus que les états 'awake' et 'asleep'.

In [ ]:
# Modifier la variable cible 'state' pour n'avoir que deux classes : 'awake' et 'asleep'
data_training['state'] = data_training['state'].replace({'w': 'awake', 'r': 'asleep', 'n': 'asleep'})
data_training.state.value_counts().plot(kind='bar', cmap='Dark2')
plt.title('Total Occurences per State')
plt.xticks(rotation=0)
plt.ylabel('Count')
plt.xlabel('State');

In [ ]:
# Séparer les caractéristiques et la variable cible
input_data = data_training.drop('state', axis=1).values
output_data = data_training['state'].values

# Garder seulement les 25 premières caractéristiques
input_data = input_data[:, :25]

# Normaliser les données d'entrée
scaler = StandardScaler()
input_data = scaler.fit_transform(input_data)

# Encoder la variable cible
encoder = LabelEncoder()
output_data = encoder.fit_transform(output_data)

# Définir KFold
kf = KFold(n_splits=3, shuffle=True, random_state=1)

### Creation du modèle MLP


In [ ]:

def create():
    model = Sequential([
        # Entrée avec les 25 caractéristiques
        Input(shape=(25,)),

        # Première couche cachée avec régularisation L2
        Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001)),

        # Deuxième couche cachée avec régularisation L2
        Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.001)),

        # Dropout pour éviter le surapprentissage, 30% des neurones sont mis à zéro durant l'entraînement
        Dropout(0.3),

        # Couche de sortie
        Dense(1, activation='sigmoid')
    ])

    # Compiler le modèle avec l'optimiseur Adam
    model.compile(
        loss='binary_crossentropy',
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        metrics=['accuracy']
    )

    return model

# Afficher le modèle
model = create()
model.summary()

### Entraînement du modèle

In [ ]:
# Stockage des métriques
train_losses = []
val_losses = []
f1_scores = []
conf_matrices = []

# Entraînement du modèle avec validation croisée
for fold, (train_index, val_index) in enumerate(kf.split(input_data)):
    X_train, X_val = input_data[train_index], input_data[val_index]
    y_train, y_val = output_data[train_index], output_data[val_index]

    model = create()

    history = model.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val))

    # Stocker les pertes
    train_losses.append(history.history['loss'])
    val_losses.append(history.history['val_loss'])

    # Prédictions sur l'ensemble de validation
    y_pred = model.predict(X_val)
    y_pred = (y_pred > 0.5).astype(int)

    # Matrice de confusion
    cm = confusion_matrix(y_val, y_pred)
    conf_matrices.append(cm)

    # Calcul du F1-score
    f1 = f1_score(y_val, y_pred)
    f1_scores.append(f1)

### Évaluation du modèle

In [ ]:
# Affichage des courbes de perte séparément par fold
for fold in range(3):
    # Courbe de perte
    plt.figure(figsize=(8, 5))
    plt.plot(train_losses[fold], label='Training Loss', color='blue')
    plt.plot(val_losses[fold], label='Validation Loss', color='red', linestyle='dashed')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title(f'Training vs Validation Loss - Fold {fold+1}')
    plt.legend()
    plt.grid()
    plt.tight_layout()  # Ajuste les espaces entre les graphiques pour éviter les chevauchements
    plt.show()

    # Matrice de confusion
    plt.figure(figsize=(5, 4))
    sns.heatmap(conf_matrices[fold], annot=True, fmt='d', cmap='Blues', xticklabels=['Asleep', 'Awake'], yticklabels=['Asleep', 'Awake'])
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title(f'Confusion Matrix - Fold {fold+1}')
    plt.tight_layout()  # Ajoute de l'espace entre la matrice et les axes
    plt.show()

# Affichage du F1-score moyen final
mean_f1 = np.mean(f1_scores)
print(f'\nF1-score moyen final : {mean_f1:.4f}')


### Évaluation du modèle sur le jeu de données de test

In [ ]:
# Chargement des données de test
data_test = pd.read_csv('EEG_mouse_data_test.csv')

In [ ]:
# Prendre les 25 premières caractéristiques de l'entraînement
columns_used_for_training = data_training.drop('state', axis=1).columns[:25]

# Sélectionner uniquement ces 25 colonnes dans les données de test
data_test_selected = data_test[columns_used_for_training]

# Normaliser les données de test avec le même scaler
X_test_final_normalized = scaler.transform(data_test_selected)

# Prédiction des classes pour les données de test
y_pred_final = model.predict(X_test_final_normalized)

# Créer un DataFrame pour les prédictions et les enregistrer dans un fichier CSV
predictions_df = pd.DataFrame(predicted_classes, columns=['Predicted_Class'])

# Enregistrer dans un fichier CSV
predictions_df.to_csv('predictions_test.csv', index=False)

# Afficher le message de confirmation
print("Les prédictions ont été enregistrées dans 'predictions_test.csv'.")

# Plot en barres de la distribution des classes prédites
plt.figure(figsize=(8, 6))
predictions_df['Predicted_Class'].value_counts().plot(kind='bar', color='darkgreen')
plt.title('Distribution des Classes Prédites dans les Données de Test')
plt.xlabel('Classe Prédite')
plt.ylabel('Nombre d\'Occurrences')
plt.xticks(rotation=0)
plt.show()